In [1]:
import my_ast
my_ast.process_prompt('./prompt-base.md', """
You play the role of a software architect, AI expert, and telecommunications expert to answer questions for me.
使用langchain 和 bingsearch 实现一个问答系统
""")


/home/weiz/.py/venvs/llm/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in LLMConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


```thinking
Hmm, this is an interesting system design question about building a QA system with LangChain and Bing Search... Let me think through this carefully.

First, I need to break down the core components needed here. We'll need LangChain as the framework to orchestrate everything, and Bing Search API to retrieve relevant information from the web. The basic flow would be: take user question -> search Bing -> process results -> generate answer.

The key elements we'll need:
- Bing Search API credentials and setup
- LangChain components for search, text processing, and response generation
- A way to structure the retrieved information
- Proper prompt engineering to get good answers

I should think about potential challenges:
- Rate limits and costs of the Bing API
- Processing and filtering the search results effectively
- Ensuring relevant and accurate answers
- Error handling and edge cases

For implementation approach, we'll want to:
1. Set up the environment and dependencies
2. 

In [1]:
from langchain_community.utilities import BingSearchAPIWrapper

search = BingSearchAPIWrapper(k=4)

search.run("python")



'Experienced programmers in any other language can pick up <b>Python</b> very quickly, and beginners find the clean syntax and indentation structure easy to learn. Whet your appetite with our <b>Python</b> 3 overview. Installer packages for <b>Python</b> on macOS downloadable from <b>python</b>.org are signed with with an Apple Developer ID Installer certificate. As of <b>Python</b> 3.11.4 and 3.12.0b1 (2023-05-23), release installer packages are signed with certificates issued to the <b>Python</b> Software Foundation (Apple Developer ID BMM5U3QVKW) ). <b>Python</b> <b>基础教程</b> Python 是一种<b>解释型</b>、面向对象、动态数据类型的高级程序设计语言。 Python 由 Guido van Rossum 于 1989 年底发明，第一个公开发行版发行于 1991 年。 像 Perl 语言一样, Python 源代码同样遵循 GPL(GNU General Public License) 协议。 使用 <b>Python</b> <b>进行计算很</b>简单，表达式语法也很简单：运算符+, -, * 和 / 按预期工作; 括号 可用于分组. 有关 Python 3 中简单数学函数的更多信息 . # 列表上的 For 循环 &gt;&gt;&gt; numbers = [2, 4, 6, 8] &gt;&gt;&gt; product = 1 &gt;&gt;&gt; for number in numbers: ...'

In [9]:
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(),override=True)
# 初始化 Bing 搜索
search = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    k=4
)

# 创建搜索工具
search_tool = Tool(
    name="Bing Search",
    description="用于搜索互联网信息的工具",
    func=search.run
)

def search(question):
    search_result = search_tool.run(question)
    return search_result

question = "什么是量子计算"
search_result = search(question)
print(search_result)


TypeError: BingSearchAPIWrapper.results() missing 1 required positional argument: 'num_results'

'Experienced programmers in any other language can pick up <b>Python</b> very quickly, and beginners find the clean syntax and indentation structure easy to learn. Whet your appetite with our <b>Python</b> 3 overview. Installer packages for <b>Python</b> on macOS downloadable from <b>python</b>.org are signed with with an Apple Developer ID Installer certificate. As of <b>Python</b> 3.11.4 and 3.12.0b1 (2023-05-23), release installer packages are signed with certificates issued to the <b>Python</b> Software Foundation (Apple Developer ID BMM5U3QVKW) ). <b>Python</b> <b>基础教程</b> Python 是一种<b>解释型</b>、面向对象、动态数据类型的高级程序设计语言。 Python 由 Guido van Rossum 于 1989 年底发明，第一个公开发行版发行于 1991 年。 像 Perl 语言一样, Python 源代码同样遵循 GPL(GNU General Public License) 协议。 使用 <b>Python</b> <b>进行计算很</b>简单，表达式语法也很简单：运算符+, -, * 和 / 按预期工作; 括号 可用于分组. 有关 Python 3 中简单数学函数的更多信息 . # 列表上的 For 循环 &gt;&gt;&gt; numbers = [2, 4, 6, 8] &gt;&gt;&gt; product = 1 &gt;&gt;&gt; for number in numbers: ...'

In [5]:
from langchain.llms import OpenAI
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
import os

# 加载环境变量
load_dotenv()

# 初始化 Bing 搜索
search = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_search_api_key=os.getenv("BING_SEARCH_API_KEY", ""),
    # k=4
)

# 创建搜索工具
search_tool = Tool(
    name="Bing Search",
    description="用于搜索互联网信息的工具",
    func=search.run
)

def search(question):
    search_result = search_tool.run(question)
    return search_result

# 设置提示模板
prompt_template = """根据以下信息回答问题：

搜索结果: {search_result}
问题: {question}

请提供详细的答案："""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_result", "question"]
)

# 创建 LLM Chain
load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")
chain = LLMChain(llm=llm, prompt=prompt)

def get_answer(question):
    try:
        # 执行搜索
        search_result = search_tool.run(question)
        
        # 生成答案
        response = chain.run({
            "search_result": search_result,
            "question": question
        })
        
        return response
    except Exception as e:
        return f"发生错误: {str(e)}"

# 使用示例
question = "什么是量子计算？"
answer = get_answer(question)
print(answer)

根据提供的信息，我可以对量子计算做如下详细解释：

量子计算是一种全新的计算模式，其核心特点如下：

1. 基本定义：
- 这是一种遵循量子力学规律调控量子信息单元进行计算的新型计算模式
- 使用亚原子粒子的物理学领域来执行复杂的并行计算
- 将原子、电子或光子等基本粒子的量子特性与量子比特结合起来，作为计算中的一种新形式的信息编码

2. 核心特征：
- 使用量子比特(qubit)作为基本运算单元，而不是传统计算机的晶体管
- 量子比特可以处于叠加态，即不仅可以是0或1，还可以同时存在多个状态
- 利用量子力学的基本原理来加速解决复杂计算的过程

3. 优势：
- 可以比传统计算机更有效地处理大量复杂的数据集
- 能够突破经典算力瓶颈
- 在某些特定计算任务上，效率远超传统计算机
- 可以解决传统计算机或超级计算机无法解决或无法快速解决的复杂问题

4. 应用原理：
- 通过量子叠加和纠缠等物理特性进行计算
- 利用专门的硬件和基于量子力学的算法
- 可以同时处理多个状态，实现并行计算

这种新型计算模式代表了计算技术的一次革命性突破，是未来计算发展的重要方向。


In [1]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")
response = llm.stream("你是哪个版本的模型")
for chunk in response:
    print(chunk.content, end="")

我是 Claude，一个由 Anthropic 开发的 AI 助手。我会努力用专业、诚实和有帮助的方式来回答你的问题。

In [4]:
import datetime
prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + """
把下面的代码进行函数和类抽取重构，使之更简洁，易扩展，易维护，代码不需要注释
新代码应该支持修改模型名称，是否使用流式
```python
from dotenv import load_dotenv, find_dotenv
import os
import datetime

load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")

prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + "1.8 和 1.11 谁大"
response = llm.stream(prompt)
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
file_path = f'sessions/temp-{timestamp}.md'

with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        print(chunk.content, end="")
        file.write(chunk.content)
```
"""
response = llm.stream(prompt)
# 获取当前时间戳
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# 定义文件路径
file_path = f'sessions/temp-{timestamp}.md'

# 将内容写入文件
with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        # print(chunk.content, end="")
        file.write(chunk.content)